# Manipulando dados no excel

Usaremos 2 bibliotecas para manipulação de dados:
 * [openpyxl](https://openpyxl.readthedocs.io/en/stable/)
 * [pandas](https://pandas.pydata.org/)

### Usaremos agora a **pandas**

In [2]:
import pandas as pd

#### Indicando o arquivo em excel
_Obs:_ É possível usar tanto extensões od tipo ".xlsx" e ".csv"

In [3]:
workSpace = pd.read_excel("data/municipio_mensal.xlsx", sheet_name="Consumo MWh")

#### Podemos ver o tamanho da planilha usando o método "shape()"

In [4]:
print(workSpace.shape)

(4512, 353)


Também podemos retirar as colunas que não nos interessam, como:
 * tipo 
 * cod_muni
 * agência
 * núcleo
 * unidade
 * cod_classe
 * município (Após a filtragem)

In [6]:
# Mantendo no DataFrame apenas os dados com município igual a Florianópolis
workSpace = workSpace.loc[workSpace["município"] == "Florianópolis"]
# Removendo as colunas que não serão utilizadas em nossa análise
workSpace = workSpace.drop(columns=['tipo', 'cod_muni', 'agência', 'núcleo', 'unidade', 'cod_classe', 'município'])

Agora possuímos apenas dados referentes ao município de Florianópolis. Porém a distribuição destes dados no DataFrame não está de forma ideal para analisá-los. Utilizaremos a função **melt()** para despivotar a tabela utilizando a coluna classe como referência, assim alterando o formato de **wide** para **long**

In [7]:
# Utilizando a feature classe como ponto de pivotamento, assim tendo o DataFrame corretamente construído
df_floripa = workSpace.melt(id_vars=['classe'])

# Renomeando as colunas restantes para refletir o que estamos analisando
df_floripa.rename(columns = {'variable':'Data','value':'MWh'}, inplace = True)

Agora vamos separar os dados de consumo em relação a sua classe, criando um DataFrame para cada uma.

In [8]:
# Separando os dados de consumo pela classe em diferentes DataFrames
df_res = df_floripa.loc[df_floripa['classe'] == 'Residencial']
df_ind = df_floripa.loc[df_floripa['classe'] == 'Industrial']
df_com = df_floripa.loc[df_floripa['classe'] == 'Comercial']
df_rur = df_floripa.loc[df_floripa['classe'] == 'Rural']
df_pub = df_floripa.loc[df_floripa['classe'] == 'Poder Público']
df_ilu = df_floripa.loc[df_floripa['classe'] == 'Iluminação Pública']
df_ser = df_floripa.loc[df_floripa['classe'] == 'Serviço Público']
df_pro = df_floripa.loc[df_floripa['classe'] == 'Próprio']

#print(df_res)

#### Filtrando ainda mais
 * Removendo a coluna "classe"
 * Somando todos os valores da coluna "MWh" que possuem as datas da coluna "Data" iguais

Como podemos perceber acima, as linhas de consumo se repetem, mesmo pertencendo a mesma classe. Assim utilizaremos a função utilizando as funções **groupby()** e **sum()**

In [9]:
df_res = df_res.groupby(['Data'], as_index=False).sum()
df_ind = df_ind.groupby(['Data'], as_index=False).sum()
df_com = df_com.groupby(['Data'], as_index=False).sum()
df_rur = df_rur.groupby(['Data'], as_index=False).sum()
df_pub = df_pub.groupby(['Data'], as_index=False).sum()
df_ilu = df_ilu.groupby(['Data'], as_index=False).sum()
df_ser = df_ser.groupby(['Data'], as_index=False).sum()
df_pro = df_pro.groupby(['Data'], as_index=False).sum()
#print(df_com)

#print(df_res["Data"])

#### Removendo a coluna "Data"

#### Renomeando a coluna "Data" para "Datetime"

In [10]:
df_res = df_res.rename(columns={"Data": "Datetime"})
df_ind = df_ind.rename(columns={"Data": "Datetime"})
df_com = df_com.rename(columns={"Data": "Datetime"})
df_rur = df_rur.rename(columns={"Data": "Datetime"})
df_pub = df_pub.rename(columns={"Data": "Datetime"})
df_ilu = df_ilu.rename(columns={"Data": "Datetime"})
df_ser = df_ser.rename(columns={"Data": "Datetime"})
df_pro = df_pro.rename(columns={"Data": "Datetime"})

2) Criando a coluna "Time"

In [11]:
df_res.insert(2, "Time", "00:00:00")
df_ind.insert(2, "Time", "00:00:00")
df_com.insert(2, "Time", "00:00:00")
df_rur.insert(2, "Time", "00:00:00")
df_pub.insert(2, "Time", "00:00:00")
df_ilu.insert(2, "Time", "00:00:00")
df_ser.insert(2, "Time", "00:00:00")
df_pro.insert(2, "Time", "00:00:00")

#### Convertendo para o formato ".csv"

1) Concatenando as colunas "Datetime", "Time" e "MWh"

In [12]:
df_res_concat = pd.concat([df_res["Datetime"], df_res["Time"], df_res["MWh"]], axis=1)
df_ind_concat = pd.concat([df_ind["Datetime"], df_ind["Time"], df_ind["MWh"]], axis=1)
df_com_concat = pd.concat([df_com["Datetime"], df_com["Time"], df_com["MWh"]], axis=1)
df_rur_concat = pd.concat([df_rur["Datetime"], df_rur["Time"], df_rur["MWh"]], axis=1)
df_pub_concat = pd.concat([df_pub["Datetime"], df_pub["Time"], df_pub["MWh"]], axis=1)
df_ilu_concat = pd.concat([df_ilu["Datetime"], df_ilu["Time"], df_ilu["MWh"]], axis=1)
df_ser_concat = pd.concat([df_ser["Datetime"], df_ser["Time"], df_ser["MWh"]], axis=1)
df_pro_concat = pd.concat([df_pro["Datetime"], df_pro["Time"], df_pro["MWh"]], axis=1)

3) Renomeando o nome da coluna de "Datetime,Time,MWh" para "Datetime,MWh"

In [13]:
df_res_concat = df_res_concat.rename(columns={"Datetime,Time,MWh": "Datetime,MWh"})
df_ind_concat = df_ind_concat.rename(columns={"Datetime,Time,MWh": "Datetime,MWh"})
df_com_concat = df_com_concat.rename(columns={"Datetime,Time,MWh": "Datetime,MWh"})
df_rur_concat = df_rur_concat.rename(columns={"Datetime,Time,MWh": "Datetime,MWh"})
df_pub_concat = df_pub_concat.rename(columns={"Datetime,Time,MWh": "Datetime,MWh"})
df_ilu_concat = df_ilu_concat.rename(columns={"Datetime,Time,MWh": "Datetime,MWh"})
df_ser_concat = df_ser_concat.rename(columns={"Datetime,Time,MWh": "Datetime,MWh"})
df_pro_concat = df_pro_concat.rename(columns={"Datetime,Time,MWh": "Datetime,MWh"})

In [21]:
print(df_res_concat)

      Datetime      Time          MWh
0   1994-01-01  00:00:00  21357.14700
1   1994-02-01  00:00:00  20782.78500
2   1994-03-01  00:00:00  19920.44200
3   1994-04-01  00:00:00  19890.39500
4   1994-05-01  00:00:00  19409.53900
..         ...       ...          ...
340 2022-05-01  00:00:00  49513.31500
341 2022-06-01  00:00:00  51371.37700
342 2022-07-01  00:00:00  50536.92089
343 2022-08-01  00:00:00  50106.64000
344 2022-09-01  00:00:00  50833.49600

[345 rows x 3 columns]


4) Removendo a vírgula entre a data e a hora

5) Convertendo no formato ".csv" e salvando os modelos filtrados.

In [19]:
# Salvando as alterações concatenadas
df_res_concat.to_csv("data/After Filters/res_municipio_mensal.csv", index=False)
df_ind_concat.to_csv("data/After Filters/ind_municipio_mensal.csv", index=False)
df_com_concat.to_csv("data/After Filters/com_municipio_mensal.csv", index=False)
df_rur_concat.to_csv("data/After Filters/rur_municipio_mensal.csv", index=False)
df_pub_concat.to_csv("data/After Filters/pub_municipio_mensal.csv", index=False)
df_ilu_concat.to_csv("data/After Filters/ilu_municipio_mensal.csv", index=False)
df_ser_concat.to_csv("data/After Filters/ser_municipio_mensal.csv", index=False)
df_pro_concat.to_csv("data/After Filters/pro_municipio_mensal.csv", index=False)

In [27]:
df = pd.read_csv("data/After Filters/res_municipio_mensal.csv")
print(df)
df["Datetime,MWh"] = df["Datetime,MWh"].str.replace(",0", " 0")
df_res_concat.to_csv("data/After Filters/res_municipio_mensal.csv", index=False)

df = pd.read_csv("data/After Filters/ind_municipio_mensal.csv")
df["Datetime,MWh"] = df["Datetime,MWh"].str.replace(",0", " 0")
df_ind_concat.to_csv("data/After Filters/ind_municipio_mensal.csv", index=False)

df = pd.read_csv("data/After Filters/com_municipio_mensal.csv")
df["Datetime,MWh"] = df["Datetime,MWh"].str.replace(",0", " 0")
df_com_concat.to_csv("data/After Filters/com_municipio_mensal.csv", index=False)

df = pd.read_csv("data/After Filters/rur_municipio_mensal.csv")
df["Datetime,MWh"] = df["Datetime,MWh"].str.replace(",0", " 0")
df_rur_concat.to_csv("data/After Filters/rur_municipio_mensal.csv", index=False)

df = pd.read_csv("data/After Filters/pub_municipio_mensal.csv")
df["Datetime,MWh"] = df["Datetime,MWh"].str.replace(",0", " 0")
df_pub_concat.to_csv("data/After Filters/pub_municipio_mensal.csv", index=False)

df = pd.read_csv("data/After Filters/ilu_municipio_mensal.csv")
df["Datetime,MWh"] = df["Datetime,MWh"].str.replace(",0", " 0")
df_ilu_concat.to_csv("data/After Filters/ilu_municipio_mensal.csv", index=False)

df = pd.read_csv("data/After Filters/ser_municipio_mensal.csv")
df["Datetime,MWh"] = df["Datetime,MWh"].str.replace(",0", " 0")
df_ser_concat.to_csv("data/After Filters/ser_municipio_mensal.csv", index=False)

df = pd.read_csv("data/After Filters/pro_municipio_mensal.csv")
df["Datetime,MWh"] = df["Datetime,MWh"].str.replace(",0", " 0")
df_pro_concat.to_csv("data/After Filters/pro_municipio_mensal.csv", index=False)

       Datetime      Time          MWh
0    1994-01-01  00:00:00  21357.14700
1    1994-02-01  00:00:00  20782.78500
2    1994-03-01  00:00:00  19920.44200
3    1994-04-01  00:00:00  19890.39500
4    1994-05-01  00:00:00  19409.53900
..          ...       ...          ...
340  2022-05-01  00:00:00  49513.31500
341  2022-06-01  00:00:00  51371.37700
342  2022-07-01  00:00:00  50536.92089
343  2022-08-01  00:00:00  50106.64000
344  2022-09-01  00:00:00  50833.49600

[345 rows x 3 columns]


KeyError: 'Datetime,Time,MWh'